Hi there! The purpose of this notebook - develop and test YOlO2PyTorch conveter before to pack it inside .py scripts. <br>
Supported YOLO version:
* YOLO v3

Based on the tutorials: https://blog.paperspace.com/how-to-implement-a-yolo-object-detector-in-pytorch/

Additional info:
* Cfg parameters meaning: https://github.com/AlexeyAB/darknet/wiki/CFG-Parameters-in-the-different-layers
* ```net``` section cfg parameters meaning: https://github.com/AlexeyAB/darknet/wiki/CFG-Parameters-in-the-%5Bnet%5D-section

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

print(torch.__version__)

1.7.0+cpu


### Helping functions

In [2]:
def parse_cfg(file_path):
    """
    Parse .cfg darknet file with neural net architecture.
    Input:
        Path to a .cfg file
    Output:
        Returns a list of blocks. Each blocks describes a block in the neural
        network to be built. Block is represented as a dictionary in the list

    """
    # Split cfg files to the lines
    cfg_file = open(file_path, 'r')
    lines = cfg_file.read().splitlines()
    lines = [x.rstrip().lstrip() for x in lines if (x != '') and ('#' not in x)]
    blocks = []
    # Pack it into list of dict
    for line in lines:
        if line[0] == '[':                             # [ symbols means new block announcement
            block = {}
            block['type'] = line[1:-1]
            blocks.append(block)
        else:
            key, value = line.split('=')
            block[key.rstrip()] = value.lstrip()       # rsplit/lsplit in case user type ' = ' instead of '='    
    return blocks

In [7]:
def create_modules(blocks):
    '''
    Create PyTorch model based on parsed darknet model blocks
    Input:
        'blocks' list from the 'parse_cfg' function
    Output:
        PyTorch ModuleList object
    '''
    # Setting up variables
    module_list = nn.ModuleList()
    prev_filters = 3               # Track previous layer depth. Initial depth - 3 (RGB image)
    output_filters = []            # Track all ???
    
    # Capture general information about a net
    net_info = blocks[0]
    
    for idx, x in enumerate(blocks[1:]):
        module = nn.Sequential()
        
        ## CONVOLUTIONAL LAYERS
        # Get info about a convolutional layer
        if x['type'] == 'convolutional':
            try:
                batch_norm = int(x['batch_normalize'])
                bias = False
            except:
                batch_norm = 0
                bias = True
            activation = x['activation']    
            filters = int(x['filters'])
            kernel_size = int(x['size'])
            stride = int(x['stride'])
            padding = int(x['pad'])
            
            # Convert padding values according to AlexeyAB's wiki page
            if padding == 1:
                padding = kenel_size // 2   
            else:
                padding = padding
                
            # Create PyTorch convolutional layer
            conv = nn.Conv2d(prev_filters, filters, kernel_size, stride, padding, bias=bias)
            module.add_module('conv_{}'.format(idx), conv)
        
            # Add BatchNorm layer
            if batch_norm:
                bn = nn.BatchNorm2d(filters)
                module.add_module('batch_norm_{}'.format(idx), bn)

            # Add Activation layer
            if activation == 'leaky':
                activ = nn.LeakyReLU(negative_slope=0.1, inplace=True)
                module.add_module('leaky_ReLu_{}'.format(idx), activ)
            else: # linear case
                pass                                                            #???
            
        ## UPSAMPLE LAYER
        
        
        ## SHORTCUT LAYER
        
        
        ## ROUTE LAYER
        
        ## YOLO LAYER
        
        ## UPDATE VARIABLES
        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)
        
    return module_list
                
            
            
    
    
    

In [5]:
FILE_PATH = './detector/cfg/yolov3.cfg'
result = parse_cfg(FILE_PATH)

In [10]:
result[1]

{'type': 'convolutional',
 'batch_normalize': '1',
 'filters': '32',
 'size': '3',
 'stride': '1',
 'pad': '1',
 'activation': 'leaky'}